In [48]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *


In [49]:
import pandas as pd

In [50]:
# load the medication data
medication = pd.read_excel('medication_data.xlsx', sheet_name = "Diabetes")
patient = pd.read_excel('medication_data.xlsx', sheet_name = 'Patients')
medication.head()

,Drug Generic Product Name,Copay,A1C Reduction,Ok to use in pregnancy,use_for_pregnancy
0,ACARBOSE TAB,5,0.005,No,0
1,BROMOCRIPTINE MESYLATE TAB,50,0.005,No,0
2,DAPAGLIFLOZIN PROP-METFORMIN HCL TAB ER 24HR,35,0.020,No,0
3,DAPAGLIFLOZIN PROPANEDIOL TAB,35,0.020,No,0
4,DULAGLUTIDE SOLN PEN-INJECTOR,35,0.015,No,0


In [51]:
# extract price and A1C reduction information from medication data
price = medication['Copay'].tolist()
A1C_reduction = medication['A1C Reduction'].tolist()
use_for_pregnancy = medication['use_for_pregnancy']

In [52]:
# extract patient information
current_A1C = patient['Current A1C Level'].tolist()
pregnancy = patient['Pregnant'].tolist()

In [53]:
# extract medication name
medication_name = medication['Drug Generic Product Name'].tolist()

In [83]:
# wrap pyomo code into the MedicationPlan function
def MedicationPlan():

  optimal_a1c = 0.07 # we need to ensure the A1C level below 7%
  minimal_a1c = 0.04 # post-treatment a1c level no less than 4%
  num_medication = len(medication) # we have 38 medication options in the dataset # i
  max_num_medication = 3 # each patient can be treated by at most 3 medications

# set model
  model = ConcreteModel()
# set DVs
  model.x = Var(range(num_medication), within = Binary)
# set objective variable
  model.obj= Objective(expr = sum(price[i] * model.x[i] for i in range(num_medication)), sense=minimize)
# set constraints
# Constraint 1: control the A1C level of each patient under 7%
  model.c1 = Constraint(expr = current_A1C - sum(A1C_reduction[i] * model.x[i] for i in range(num_medication)) <= optimal_a1c)
# Constraint 2: each patient can be treated at most three medications in case of hypoglycemia risk
  model.c2 = Constraint(expr = sum(model.x[i] for i in range(num_medication))<= max_num_medication)
# Constraint 3: post-treatment A1c level higher than 4%
  model.c3 = Constraint(expr = current_A1C - sum(A1C_reduction[i] * model.x[i] for i in range(num_medication)) >= minimal_a1c)
# Constraint 4: pregnant patient can only be treated with specific medication
  def pregnancy_constraint_rule(model):
    sum_1 = sum(model.x[i] for i in range(num_medication) if pregnancy == 1 and use_for_pregnancy[i] == 1)
    sum_2 = sum(model.x[i] for i in range(num_medication) if pregnancy == 1)
    return sum_1 - sum_2 >= 0

  opt = SolverFactory('cbc')
  opt.solve(model)

  objective = model.obj()
  medication_plan_matrix = [medication_name[i] for i in range(num_medication) if model.x[i]() > 0]
  return objective, medication_plan_matrix


In [85]:
# use MedicationPlan function to find solution for patient k
k=2 # use k to index rows in patients data
rowdata = patient.iloc[k,1:].values.tolist()
current_A1C = rowdata[0] # extract the current A1C value of patient k
pregnancy = rowdata[2] # extract the pregnancy information about patients
MedicationPlan()



(5.0, ['GLIPIZIDE-METFORMIN HCL TAB '])

In [56]:
#loop
outputs = []
for i in range(len(patient)):
  rowdata = patient.iloc[i,1:].values.tolist()
  current_A1C = rowdata[0]
  outputs.append(MedicationPlan())
patient['solution'] = outputs

  - termination condition: infeasible
  - message from solver: <undefined>


In [57]:
print(patient)

   Patient  Current A1C Level Gender   Pregnant  \
0        1              0.135       M         0   
1        2              0.085       F         1   
2        3              0.090       F         0   
3        4              0.102       M         0   
4        5              0.112       M         0   
5        6              0.129       M         0   

                                            solution  
0  (35.0, [INSULIN GLARGINE-LIXISENATIDE SOL PEN-...  
1                  (5.0, [GLYBURIDE-METFORMIN TAB ])  
2                  (5.0, [GLYBURIDE-METFORMIN TAB ])  
3  (15.0, [GLYBURIDE-METFORMIN TAB , METFORMIN HC...  
4                                            (0, [])  
5  (35.0, [INSULIN GLARGINE-LIXISENATIDE SOL PEN-...  


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
patient.to_csv('/content/drive/MyDrive/solution.csv', index = False)